In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import glob
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Conv3D,MaxPooling3D,Dropout,Flatten,LeakyReLU,BatchNormalization,Conv2D,MaxPooling2D
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm
import zipfile


In [0]:
comofod='http://www.vcl.fer.hr/comofod/comofod_small.rar'
zip_dir = tf.keras.utils.get_file('comofod_small.rar', origin=comofod)
zip_dir_base = os.path.dirname(zip_dir)
!pip install patool
import patoolib
os.chdir(zip_dir_base)
patoolib.extract_archive("comofod_small.rar", outdir=zip_dir_base)

In [0]:
auimg=glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*_O_*')
tpimg=glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*_F_*')
print(len(auimg))
print(len(tpimg))

target=zip_dir_base + '/Authentic'
os.makedirs(target)
target2=zip_dir_base + '/Tampered'
os.makedirs(target2)

for i in auimg:
  shutil.move(i, target)
  
for i in tpimg:
  shutil.move(i, target2)


In [0]:
classes=['Authentic', 'Tampered']
for cl in classes:
  img_path = os.path.join(zip_dir_base, cl)
  images = glob.glob(img_path + '/*')
  print("{}: {} Images".format(cl, len(images)))
  train, val = images[:round(len(images)*0.7)], images[round(len(images)*0.7):]

  for t in train:
    if not os.path.exists(os.path.join(zip_dir_base, 'train', cl)):
      os.makedirs(os.path.join(zip_dir_base, 'train', cl))
    shutil.move(t, os.path.join(zip_dir_base, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(zip_dir_base, 'val', cl)):
      os.makedirs(os.path.join(zip_dir_base, 'val', cl))
    shutil.move(v, os.path.join(zip_dir_base, 'val', cl))
    
train_dir = os.path.join(zip_dir_base, 'train')
val_dir = os.path.join(zip_dir_base, 'val')

In [0]:
def mfr(im):
  medi=cv2.medianBlur(im,5)
  medi2=cv2.medianBlur(medi,5)
  medi=cv2.subtract(medi2, medi)
  medi=medi[..., np.newaxis]
  
  avg = cv2.blur(im,(5,5))
  avg = cv2.subtract(avg,im)
  avg = avg[..., np.newaxis]
  
  k=np.array([[0,0,0,0,0], [0,-1,2,-1,0], [0,2,-4,2,0], [0,-1,2,-1,0], [0,0,0,0,0]], np.int32)
  k1=k/4
  srm1=cv2.filter2D(im, -1, k1)
  srm1 = srm1[...,np.newaxis]
  
  k=np.array([[-1,2,-2,2,-1], [2,-6,8,-6,2], [-2,8,-12,8,-2], [2,-6,8,-6,2], [-1,2,-2,2,-1]], np.int32)
  k2=k/12
  srm2=cv2.filter2D(im, -1, k2)
  srm2 = srm2[...,np.newaxis]
  
  k=np.array([[0,0,0,0,0], [0,0,0,0,0], [0,1,-2,1,0], [0,0,0,0,0], [0,0,0,0,0]], np.int32)
  k3=k/2
  srm3=cv2.filter2D(im, -1, k3)
  srm3=srm3[...,np.newaxis]
  
  imge=np.concatenate((medi,avg), axis=2)
  imge=np.concatenate((imge,srm1), axis=2)
  imge=np.concatenate((imge,srm2), axis=2)
  imge=np.concatenate((imge,srm3), axis=2)
  
  
  imge=imge[..., np.newaxis]
  return imge

train_image_generator = ImageDataGenerator(preprocessing_function=mfr,
                                           featurewise_center=True,
                                           featurewise_std_normalization=True)
train_data_gen = train_image_generator.flow_from_directory(batch_size=32, 
                                                           directory=train_dir, 
                                                           shuffle=True,
                                                           target_size=(128,128,5),
                                                           class_mode='sparse',
                                                           color_mode='grayscale')
val_image_generator = ImageDataGenerator(preprocessing_function=mfr,
                                         featurewise_center=True,
                                         featurewise_std_normalization=True)
val_data_gen = val_image_generator.flow_from_directory(batch_size=32, 
                                                       directory=val_dir,
                                                       
                                                       target_size=(128,128,5),
                                                       class_mode='sparse', 
                                                       color_mode='grayscale')

In [0]:
ini=tf.contrib.layers.variance_scaling_initializer(factor=2.0, mode='FAN_IN', uniform=False)

model=Sequential()
model.add(Conv3D(64,(1,1,5), strides=1,  activation='relu', input_shape=(128,128,5,1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D((3,3), strides=2, padding='same'))

model.add(Conv3D(64,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(LeakyReLU())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

model.add(Conv3D(128,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

#model.add(LeakyReLU())
model.add(Conv3D(256,(3,3,1),padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))
#model.add(LeakyReLU())

model.add(Conv3D(256,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(LeakyReLU())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

model.add(Conv3D(512,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(LeakyReLU())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

#model.add(Dropout(0.5))
#model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

#model.add(Dropout(0.5))
#model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

model.add(Dense(2, activation='softmax'))

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model.fit_generator(
                              train_data_gen,
                              epochs=120,
                              validation_data=val_data_gen)